In [1]:
from openalgo import api
import pandas as pd
from datetime import datetime, timedelta
import time
import os

# Initialize the API client
client = api(api_key='917dd42d55f63ae8f00117abfbe5b05465fc3bd76a3efbee3be7c085df0be579', host='http://127.0.0.1:5000')

# Fetch historical data for SBIN
def fetch_and_save_historical_data(client: api, symbol, exchange, interval, start_date, end_date, chunk_size, max_retries=5, retry_delay=10):
    """
    Fetch historical data in chunks and save to CSV file
    
    Parameters:
    -----------
    client : api
        API client instance
    symbol : str
        Trading symbol
    exchange : str
        Exchange name
    interval : str
        Time interval for data
    start_date : str
        Start date in YYYY-MM-DD format
    end_date : str
        End date in YYYY-MM-DD format
    chunk_size : int
        Number of days to fetch in each chunk
    max_retries : int, optional
        Maximum number of retry attempts
    retry_delay : int, optional
        Delay between retries in seconds
        
    Returns:
    --------
    pd.DataFrame
        Combined historical data
    """
    
    
    # Create data directory if it doesn't exist
    data_dir = "data"
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        
    # Convert dates to datetime
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Initialize empty list to store chunks
    chunks = []
    
    # Fetch data in chunks
    current_start = start_dt
    while current_start < end_dt:
        current_end = min(current_start + timedelta(days=chunk_size), end_dt)
        
        # Retry logic
        for attempt in range(max_retries):
            try:
                chunk = client.history(
                    symbol=symbol,
                    exchange=exchange,
                    interval=interval,
                    start_date=current_start.strftime("%Y-%m-%d"),
                    end_date=current_end.strftime("%Y-%m-%d")
                )
                
                if chunk is not None and not chunk.empty:
                    chunks.append(chunk)
                    # Save chunk to CSV
                    filename = f"{data_dir}/{symbol}_{exchange}_{interval}_{current_start.strftime('%Y%m%d')}_{current_end.strftime('%Y%m%d')}.csv"
                    chunk.to_csv(filename)
                    print(f"Saved chunk to {filename}")
                    break
                    
            except Exception as e:
                if attempt == max_retries - 1:
                    print(f"Failed to fetch data after {max_retries} attempts: {str(e)}")
                    raise
                print(f"Attempt {attempt + 1} failed, retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
        
        current_start = current_end
    
    # Combine all chunks
    if chunks:
        df = pd.concat(chunks, axis=0)
        # Save combined data
        combined_filename = f"{data_dir}/{symbol}_{exchange}_{interval}_{start_date}_{end_date}.csv"
        df.to_csv(combined_filename)
        print(f"Saved combined data to {combined_filename}")
        return df
    else:
        print("No data fetched")
        return None

# Example usage:
df = fetch_and_save_historical_data(
    client=client,
    symbol="SBIN",
    exchange="NSE",
    interval="1m",
    start_date="2017-07-03",
    end_date=f"{pd.Timestamp.now().strftime("%Y-%m-%d")}",
    chunk_size=30  # Fetch 10 days at a time
)

df

Saved chunk to data/SBIN_NSE_1m_20170703_20170802.csv
Saved chunk to data/SBIN_NSE_1m_20170802_20170901.csv
Saved chunk to data/SBIN_NSE_1m_20170901_20171001.csv
Saved chunk to data/SBIN_NSE_1m_20171001_20171031.csv
Saved chunk to data/SBIN_NSE_1m_20171031_20171130.csv
Saved chunk to data/SBIN_NSE_1m_20171130_20171230.csv
Saved chunk to data/SBIN_NSE_1m_20171230_20180129.csv
Saved chunk to data/SBIN_NSE_1m_20180129_20180228.csv
Saved chunk to data/SBIN_NSE_1m_20180228_20180330.csv
Saved chunk to data/SBIN_NSE_1m_20180330_20180429.csv
Saved chunk to data/SBIN_NSE_1m_20180429_20180529.csv
Saved chunk to data/SBIN_NSE_1m_20180529_20180628.csv
Saved chunk to data/SBIN_NSE_1m_20180628_20180728.csv
Saved chunk to data/SBIN_NSE_1m_20180728_20180827.csv
Saved chunk to data/SBIN_NSE_1m_20180827_20180926.csv
Saved chunk to data/SBIN_NSE_1m_20180926_20181026.csv
Saved chunk to data/SBIN_NSE_1m_20181026_20181125.csv
Saved chunk to data/SBIN_NSE_1m_20181125_20181225.csv
Saved chunk to data/SBIN_NSE

KeyboardInterrupt: 